In [0]:
from pyspark.sql import SparkSession

# Parâmetros 
year = 2023
month_from = 1
month_to = 4

spark = SparkSession.builder.getOrCreate()
spark.sql("USE CATALOG nyc_taxi")
spark.sql("USE SCHEMA data_area")



In [0]:
## Evolução diária volume, receita e média móvel 7 dias
spark.sql(f'''
WITH base AS (
  SELECT
    dt,
    qtd_viagens,
    receita_total,
    ticket_medio
  FROM nyc_taxi.data_area.v_gold_fat_diario
  WHERE dataset_year = {year}
    AND dataset_month BETWEEN {month_from} AND {month_to}
)
SELECT
  dt,
  qtd_viagens,
  receita_total,
  ticket_medio,
  AVG(qtd_viagens)   OVER (ORDER BY dt ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS mm7_viagens,
  AVG(receita_total) OVER (ORDER BY dt ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS mm7_receita
FROM base
ORDER BY dt
''').show(50, truncate=False)

## Apresenta as movimentações adiarias, não tem tanta variações e mantem uma tendencia bem equilibrada.

In [0]:
## Mix de pagamento. Share de corridas e de receita"

spark.sql(f'''
WITH agg AS (
  SELECT
    payment_desc,
    SUM(qtd_viagens)   AS viagens,
    SUM(receita_total) AS receita
  FROM nyc_taxi.data_area.v_gold_fact_payment
  WHERE dataset_year = {year}
    AND dataset_month BETWEEN {month_from} AND {month_to}
  GROUP BY payment_desc
),
tot AS (
  SELECT
    SUM(viagens) AS tot_viagens,
    SUM(receita) AS tot_receita
  FROM agg
)
SELECT
  a.payment_desc,
  a.viagens,
  a.receita,
  ROUND(a.viagens / t.tot_viagens * 100, 2) AS pct_viagens,
  ROUND(a.receita / t.tot_receita * 100, 2) AS pct_receita
FROM agg a CROSS JOIN tot t
ORDER BY receita DESC
''').show(truncate=False)

## O perfil de pagamento dominante é o cartão de credito, justamente pea facilidade de pagamento por app e aproximação.

In [0]:
## Perfil por hora do dia, quando há mais corridas/receita
spark.sql(f'''
SELECT
  hour,
  SUM(qtd_viagens)   AS viagens,
  SUM(receita_total) AS receita,
  ROUND(AVG(ticket_medio), 2) AS ticket_medio_medio
FROM nyc_taxi.data_area.v_gold_fact_hour
WHERE dataset_year = {year}
  AND dataset_month BETWEEN {month_from} AND {month_to}
GROUP BY hour
ORDER BY SUM(qtd_viagens), SUM(receita_total) DESC
''').show(50, truncate=False)

## Pelos dados podemos entender que nem sempre a quantidade de corridas vai gerar um ticket maior, mas os horarios considerados de madrugada, onde há baixa movimentação, custam mais caro.